# Analytathon 3 

## Executive summary
The purpose of our project was to develop a model to predict prices by analysing market trends, which could help Energia purchase electricity at the best price among the 5 markets. 


To make predictions, the model was trained on a dataset of historical sequence data. During training, the model learns to recognize patterns in the data and use them to make predictions about future values in the sequences. Once the model has been trained, it can be used to make predictions on new and unseen sequences, to help identify the optimal market to purchase electricity at 30-minute intervals for D+1.


However, because of the complexity of the task and time constraints, we only managed to build a base model, that could give predictions but still could be improved. Therefore, further studies need to be carried out in order to explore how the model can be optimized to obtain a better prediction. 


## Introduction
Energia Group is an Irish energy company that provides electricity and gas to customers in NI and ROI. Energia decided to buy electricity across 5 markets, and they decided to buy the electricity at the best price to optimise the business. The purpose of this report is to develop a Long Short Term Memory (LSTM) model to predict prices by analysing market trends, to guide Energia buying electricity at the lowest price. This report will also show our prediction of our model on the last day to demonstrate the performance of our model. 

The report has been organised in the following way. We begins by focusing on organising the project into manageable tasks, followed by our metrics of success.It will then go on discussing data exploration in more detail. The third part is concerned with the methodology used for this study, we will explore the construction of LSTM model and the application of LSTM model on market price forecasting. Finally we will give our recommendations and discuss the potential limitations of our work. 

Due to time constraints and the complication of the project,  this project could not provide a perfect model to guide the purchase. We have only managed to build a model that can make predictions based on historical time series of the data. 


## Data Processing

For this project, we have four variables that will affect the market price, which are demand, wind, gas price and electricity price. In the “prices_niv” file, the prices for all five markets of each 30-minute interval of everyday and national imbalance volume(NIV) are saved. We imported these five datasets into a jupyter notebook. Since some of the datasets are started from the second row and column, we formatted all the datasets immediately after importing by removing blank rows and blank columns. 

We filter out the missing rows in “demand” dataset, insert the consequent time and date according its position, and set 0 to the demand variables, so now each data date and time period is continuous and uninterrupted.

After inserting the missing rows,  we join all the datasets and filter the result dataframe by removing data whose date is after March 31, 2023. This is because the “prices_niv” stops at this data, to make sure we have all four variables for each slot to train the model, we choose to stop at this date. We convert all the data types to numeric as well in this step. The dataframe that merged all the data and contained all columns is called “combined”. 

### Load libraries

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# print all output from cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all";

### Import datasets

In [3]:
# load datasets
gas_prices = pd.read_excel("C:/Users/tomed/Downloads/Historic Gas Prices.xlsx")
wind = pd.read_excel("C:/Users/tomed/Downloads/Wind and Demand.xlsx",  sheet_name='WIND')
demand = pd.read_excel("C:/Users/tomed/Downloads/Wind and Demand.xlsx",  sheet_name='DEMAND')
electricity_prices = pd.read_excel("C:/Users/tomed/Downloads/GB Prices.xlsx")
prices_niv = pd.read_excel("C:/Users/tomed/Downloads/Prices and NIV.xlsx")


### Fix data formatting issues

In [4]:
# prices_niv formatting
# Correct columns and drop NaN column at index position [0] upon import

prices_niv.columns = prices_niv.iloc[0]
prices_niv = prices_niv.drop(prices_niv.index[0])
prices_niv.drop(prices_niv.columns[[0]], axis = 1, inplace=True)
# prices_niv.head()

# wind formatting
# Correct columns and drop NaN column at index position [0] upon import

wind.columns = wind.iloc[0]
wind = wind.drop(wind.index[0])
wind.drop(wind.columns[[0]], axis = 1, inplace=True)
# wind.head()

# demand formatting
# Correct columns and drop NaN column at index position [0] upon import

demand.columns = demand.iloc[0]
demand = demand.drop(demand.index[0])
demand.drop(demand.columns[[0]], axis = 1, inplace=True)
# demand.head()

In [ ]:
demand.iloc[21601:21610]
demand.shape

In [6]:
# 1. insert missing rows into demand

#insert row in between index position 21604 and 21605
demand.loc[21604.5] = '27/03/2022', '27/03/2022', "01:00:00", 0

#sort index
demand = demand.sort_index().reset_index(drop=True)

# 2. insert missing rows into demand

#insert row in between index position 21604 and 21605
demand.loc[21604.5] = '27/03/2022', '27/03/2022', "01:30:00", 0

#sort index
demand = demand.sort_index().reset_index(drop=True)


In [ ]:
demand.iloc[21601:21610]
demand.shape

In [3]:
# check shapes
prices_niv.shape
wind.shape
demand.shape

In [4]:
# check null values
demand.isnull().sum()
wind.isnull().sum()

### Combine dataframes

##### Join demand and wind

In [5]:
# use demand to join others

# Join wind
demand_wind = pd.merge(demand, wind, how = "left",on = ["Trade Date", "Start Date", "Start Time 30 Minute Period"])
demand_wind.tail(1)
demand_wind.isnull().sum()

demand_wind["Start Date"] = pd.to_datetime(demand_wind["Start Date"], format="%d/%m/%Y")
# Drop dates after 31/3/2023
demand_wind_1 = demand_wind
demand_wind = demand_wind_1[demand_wind_1['Start Date'] <= "2023-03-31"]
demand_wind.tail(1)

demand_wind.shape
demand_wind.isnull().sum()


##### Join demand and electricity_prices

In [7]:
## Join electricity_prices

# Rename heading to enable easy merge
electricity_prices.rename({'Start Time': 'Start Time 30 Minute Period'}, axis=1, inplace=True)
# Merge using demand which has the full date range 
demand_electricity = pd.merge(demand, electricity_prices, how = "left", on = ["Trade Date", "Start Time 30 Minute Period"])
# Drop extra titles
demand_electricity.drop(["Start Date Time", "Demand (MW)"], axis=1, inplace = True)
demand_electricity.tail(1)
demand_electricity.isnull().sum()

demand_electricity["Start Date"] = pd.to_datetime(demand_electricity["Start Date"], format="%d/%m/%Y")
# Drop dates after 31/3/2023
demand_electricity_1 = demand_electricity
demand_electricity = demand_electricity_1[demand_electricity_1['Start Date'] <= "2023-03-31"]
demand_electricity.tail(1)

demand_electricity.shape
demand_electricity.isnull().sum()

##### Join demand and gas

In [8]:
## Join gas

# Get a column of the correct dates
TradeDates = demand[["Trade Date"]].copy()
TradeDates.drop_duplicates(inplace = True)
TradeDates.reset_index(drop=True, inplace = True)

# Create column to join on and drop existing date
gas_prices["Trade Date"] = TradeDates
# Merge
demand_gas_half_hour= pd.merge(gas_prices, demand, how='right', left_on='Trade Date', right_on='Trade Date')
demand_gas_half_hour.drop(["Date", "Demand (MW)"], axis=1, inplace = True)
demand_gas_half_hour.tail(1)
demand_gas_half_hour.isnull().sum()


demand_gas_half_hour["Start Date"] = pd.to_datetime(demand_gas_half_hour["Start Date"], format="%d/%m/%Y")
# Drop dates after 31/3/2023
demand_gas_half_hour_1 = demand_gas_half_hour
demand_gas_half_hour = demand_gas_half_hour_1[demand_gas_half_hour_1['Start Date'] <= "2023-03-31"]
demand_gas_half_hour.tail(1)

demand_gas_half_hour.shape
demand_gas_half_hour.isnull().sum()


In [9]:
# set empty rows of IDA1 and IDA2 markets to 0 so that we can later join with demand dataframe and interpolate only the 8 NaN values we get
prices_niv.isnull().sum()

# replce NaNs with 0
prices_niv.fillna(0, inplace=True)

# validate
prices_niv.isnull().sum()

##### Join demand and price_niv

In [10]:
## Join price_niv
demand_price_niv = pd.merge(demand, prices_niv, how = "left",on = ["Trade Date", "Start Date", "Start Time 30 Minute Period"])
demand_price_niv.drop(["Demand (MW)"], axis=1, inplace = True)
demand_price_niv.tail(1)
demand_price_niv.isnull().sum()


demand_price_niv["Start Date"] = pd.to_datetime(demand_price_niv["Start Date"], format="%d/%m/%Y")
# Drop dates after 31/3/2023
demand_price_niv_1 = demand_price_niv
demand_price_niv = demand_price_niv_1[demand_price_niv_1['Start Date'] <= "2023-03-31"]
demand_price_niv.tail(1)

demand_price_niv.shape
demand_price_niv.isnull().sum()


In [11]:
#check shapes after merge
demand_wind.shape
demand_electricity.shape
demand_gas_half_hour.shape
demand_price_niv.shape


##### Final 'combined' dataframe

In [12]:
# Combine all the dataframes
combined = pd.merge(demand_price_niv, demand_gas_half_hour, on=["Start Date", "Trade Date", "Start Time 30 Minute Period"]
                    ).merge(demand_electricity, on=["Start Date", "Trade Date", "Start Time 30 Minute Period"]
                    ).merge(demand_wind, on=["Start Date", "Trade Date", "Start Time 30 Minute Period"])

combined["Trade Date"] = pd.to_datetime(combined["Trade Date"], format="%d/%m/%Y")
combined.shape
combined.isnull().sum()

### Interpolation

We now have variables for each day and every time slot. Yet, there are still some rows that include 0 or NAN. These are the missing values that will impact the results and we need to replace them with a reasonable value. Since the interpolation could only be applied to 0 but not NAN, NAN was converted to 0 first. Apply `pd.interpolate()` to each variable column of “combined” and set the parameter `method = ‘pad’`. This parameter will copy the value just before the missing entry, this is the most sensible method among all the methods. 

In [13]:
# check combined dataframe column data types
combined.info()

In [14]:
# convert dtypes
combined['DAM Market Price (€/MWh)'] = pd.to_numeric(combined['DAM Market Price (€/MWh)'])
combined['IDA1 Market Price (€/MWh)'] = pd.to_numeric(combined['IDA1 Market Price (€/MWh)'])
combined['IDA2 Market Price (€/MWh)'] = pd.to_numeric(combined['IDA2 Market Price (€/MWh)'])
combined['IDA3 Market Price (€/MWh)'] = pd.to_numeric(combined['IDA3 Market Price (€/MWh)'])
combined['BM Market Price (€/MWh)'] = pd.to_numeric(combined['BM Market Price (€/MWh)'])
combined['Market Net Imbalance Volume (MW)'] = pd.to_numeric(combined['Market Net Imbalance Volume (MW)'])
combined['Demand (MW)'] = pd.to_numeric(combined['Demand (MW)'])
combined['Actual Wind (MW)'] = pd.to_numeric(combined['Actual Wind (MW)'])

# check again
combined.info()

In [15]:
# check for missing values
combined.isna().sum()
combined.isnull().sum()

In [16]:
# perform interpolation for missing values
# method='pad' will copy value just before the missing entry - made more sense when comparing other methods

combined['DAM Market Price (€/MWh)'] = combined['DAM Market Price (€/MWh)'].interpolate(method='pad')
combined['IDA1 Market Price (€/MWh)'] = combined['IDA1 Market Price (€/MWh)'].interpolate(method='pad')
combined['BM Market Price (€/MWh)'] = combined['BM Market Price (€/MWh)'].interpolate(method='pad')
combined['Market Net Imbalance Volume (MW)'] = combined['Market Net Imbalance Volume (MW)'].interpolate(method='pad')
combined['Actual Wind (MW)'] = combined['Actual Wind (MW)'].interpolate(method='pad')

# # IDA2 and IDA2 only the 8 rows should be interpolated (we dealt with this by converting all missing values into 0 earlier so that only new rows remain as NaN)
combined['IDA2 Market Price (€/MWh)'] = combined['IDA2 Market Price (€/MWh)'].interpolate()
combined['IDA3 Market Price (€/MWh)'] = combined['IDA3 Market Price (€/MWh)'].interpolate()
# NOTE: 8 missing rows at 1 and 1:30 => IDA2 and IDA3 markets are closed. Might as well replace missing values in these columns with 0

In [17]:
# check for NaNs
combined.isna().sum()

### Build new column Net Demand

To help with training, we created a new column `net demand` that holds the variables created by the given variable, which represents the difference between electricity demand and wind power generation.

In [18]:
# Build net demand column
combined["Net Demand"] = combined["Demand (MW)"] - combined["Actual Wind (MW)"]
combined.head()
combined.shape

## Data Exploration 
In the data exploration, we worked out how each variable changes along the time, which is the univariate analysis; how each variable is related with others or its past data, which is the multivariate analysis; and outlier detection. We visualized time series data to achieve the above objectives. These visualization graphs can provide informative insights to specify time-related features, such as trends, cycles, and seasonality, that may affect the model selection (Brownlee, 2019).

### Univariate analysis

##### Wind & Demand Plotted By Date Across the Entire Dataset

Here we first plotted the demand, wind and net demand in a line plot, shows how the values changed with the time. From the plots, we can clearly see the existence of cycles, roughly one year at a time, with demand and wind being more cyclical and net demand being essentially flat, because it is the difference between the two above, but there is still some small downward arc, because the gap is larger near the beginning and end of the cycle.  

In [1]:
# Create copy as the index is being changed
demand_wind_graph= combined.copy()

demand_wind_graph = demand_wind_graph.drop(columns=['Trade Date', 'Start Time 30 Minute Period', 'DAM Market Price (€/MWh)', 'IDA1 Market Price (€/MWh)', 'IDA2 Market Price (€/MWh)', 'IDA3 Market Price (€/MWh)',
                                                    'BM Market Price (€/MWh)', 'Market Net Imbalance Volume (MW)', 'Gas Price £/Therm', 'GB Price (€/MWh)'])
    
# Set date as index so it's x axis by default
demand_wind_graph.set_index('Start Date', inplace = True)

# axes = demand_wind_graph.plot(sharex='all', subplots=True, title=['Forecast Wind Production', 'Forecast Demand', "Net Demand"], figsize=(20,7.5))
axes = demand_wind_graph.plot(sharex='all', subplots=True,  figsize=(20,7.5))

axes[0].xaxis.set_tick_params(labelbottom=True)
axes[0].set_ylabel("Demand MWh")

axes[1].xaxis.set_tick_params(labelbottom=True)
axes[1].set_ylabel("Wind Production MWh")

axes[2].set_ylabel("Net Demand (Demand - Wind)")
axes[2].set_xlabel("Date Period")

#Adjust plots to create gaps for legibility
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.6,
                    hspace=0.6)
plt.show();

##### Summary Statistics for Forecast Wind Production & Demand By Month

In [24]:
# Splits demand and wind by year based on index
#Runs from 2300-2300

d_w_2021= demand_wind.iloc[0:17520]
d_w_2022= demand_wind.iloc[17520:35039]
d_w_2023= demand_wind.iloc[35039:]

In [2]:
d_w_2021_sum_stats = d_w_2021.copy()
d_w_2021_sum_stats['month'] = pd.DatetimeIndex(d_w_2021_sum_stats['Start Date'], dayfirst=True).month
d_w_2021_sum_stats.groupby("month").describe()

##### Gas & Electricity Price Plotted

Next we plot the gas and electricity price.Even if these two graphs do not have exactly the same shape, we can still observe that prices show great volatility between October 2021 and April 2022. There is another peak between July to October of 2022. The price increased again and reached peak at around the end of the 2022. The trend and the position that has the peak values are nearly the same. This suggests that gas and electricity prices always have similar movements, which could be due to some reason or they are affected in the same way.

In [26]:
# Create a merged DF
gas_vs_elec = pd.merge(demand_electricity, demand_gas_half_hour, on=["Start Date", "Trade Date", "Start Time 30 Minute Period"])


In [2]:
gas_vs_elec_graph = gas_vs_elec.copy()

gas_vs_elec_graph.set_index('Start Date', inplace = True)
#plot
axes = gas_vs_elec_graph.plot(sharex='all', subplots=True, title=['GB Price (€/MWh) from 2020-2023', 'Gas Price £/Therm from 2020-2023'], figsize=(20,7.5))


axes[0].xaxis.set_tick_params(labelbottom=True)
axes[0].set_ylabel("Cost in € per MWh")

axes[1].xaxis.set_tick_params(labelbottom=True)
axes[1].set_ylabel("Cost in £ per Therm")
axes[1].set_xlabel("Date Period")

#Adjust plots to create gaps for legibility
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.6,
                    hspace=0.6)
plt.show();

##### Summary Statistics for Gas & Electricity by Year 

In [19]:
gas_vs_elec_stats = gas_vs_elec.copy()

gas_vs_elec_stats['year'] = pd.DatetimeIndex(gas_vs_elec_stats['Start Date'], dayfirst=True).year
gas_vs_elec_stats.groupby("year").describe()

<!-- ##### Plotting Markets Across the Dataset -->

Then we ploted the markets prices. The plots include 5 market and the Market Net Imbalance Volume (NIV).The plot shows the fluctuation of energy prices and NIV over time, which can be useful for understanding the trends and making informed decisions regarding energy consumption and cost management. The plot also provides a visual representation of the relationship between the different energy prices and the Market Net Imbalance Volume over the selected time period.

In [3]:
#Plotting each market across the data set

prices_and_niv_graph = demand_price_niv.copy()

prices_and_niv_graph.set_index('Start Date', inplace = True)

#plot
axes = prices_and_niv_graph.plot(sharex='all', subplots=True, title=['DAM Market Price (€/MWh)',
                                                                     'IDA1 Market Price (€/MWh)',
                                                                     "IDA2 Market Price (€/MWh)",
                                                                     "IDA3 Market Price (€/MWh)",
                                                                     "BM Market Price (€/MWh)",
                                                                     "Market Net Imbalance Volume (MW)"], 
                                 figsize=(20,7.5))


#Adjust plots to create gaps for legibility
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.6,
                    hspace=0.6)
plt.show();

If we increase the size of each plot to obtain a bigger and more detailed plot, we can observe that nearly all five markets share the same trend, they increase and decrease with similar slope and reach peak around the same time. This means the price of these five markets are actually closely correlated with each other, and we will talk about this later in the multivariate analysis secton. 

### Outlier Detection

Outliers are data points that lie far away from the majority and dramatically deviates from other observations in a dataset(2023). They are usually considerably larger or smaller than other values (Bonthu, 2023). The existence of outliers may have an impact on the result and  could result from things like measurement errors or odd occurrences.  

<!-- ##### Demand -->

The graph shows the trend of the electricity demand over time, with each data point representing the demand for electricity at a specific point in time.The graph can help to identify patterns in electricity demand, such as seasonal fluctuations or changes in demand due to specific events as well as detect for outliers. Since we know the outliers are usually larger or smaller than other values, and from this graph we observed that the outliers are all in the lower part of the graph, so we try to detect outliers by finding values that is the smallest of all numbers. 

In [4]:
# plot Demand over time
combined.plot('Start Date', 'Demand (MW)', kind='line', figsize=(15,3));

By using the 'nsmallest()' method to display the "Demand (MW)" column's 10 smallest values, the code may spot outliers. These 10 examples had the lowest demand in comparison to the rest of the dataset. These 10 data points reflect an extremely low demand compared to the rest of the dataset, and outliers are data points that drastically depart from the rest of the data.

In [20]:
# Get 10 smallest values for demand
combined.nsmallest(10, 'Demand (MW)')

<!-- ##### Gas -->

We plot gas to check for the outliers as well. We can see some values that are much higher than the rest of the data, indicating a potential anomaly or unusual event, hence these values are considered outliers. 

In [5]:
# plot Gas price over time
combined.plot('Start Date', 'Gas Price £/Therm', kind='line', figsize=(15,3));

Similar to above, we use the `nlargest()` function to get the 10 maximum values of the gas price column and treat them as anomalies.

In [21]:
# Get 10 largest values for gas price
combined.nlargest(10, 'Gas Price £/Therm')

<!-- ##### Electricity -->

As what we did before, we plot the electricity prices and we can observe that there are some abnormal values in the first season of 2021, from October 2021 to February 2022, and around December 2022. All these values are significantly larger than others. So we keep using `nlargest()` function to get the five largest values as anomalies. 

In [6]:
# plot GB price over time
combined.plot('Start Date', 'GB Price (€/MWh)', kind='line', figsize=(15,3));


In [22]:
# Get 5 largest values for GB price/MWh
combined.nlargest(5, 'GB Price (€/MWh)')

In [23]:
# comparing prices with that of 2021
combined[combined['Start Date'] == "2021-12-12"].head(3)

### Multivariate Analysis

##### Correlation Matrix For Variables of Interest

In [8]:
import seaborn as sns
sns.heatmap(combined.corr(),annot=True, cbar=False, cmap='Blues', fmt='.1f');

### Autocorrelation

We first work on the autocorrelation of each variable with `pd.autocorr()` function, this will look into the relation of past data and future data. The parameter `lag` means the period of time that compare with itself, more specifically is the number of month.  We selected 4 different numbers represents the autocorrelation between every month, season, half year and year.  
- lag = 1: calculate the correlation between month M and month M-1. 
- lag = 3: calculate the correlation between month M and month M-3, which is every season
- lag = 6: calculate the correlation between month M and month M-6, which is every half year
- lag = 12: calculate the correlation between month M and month M-12, which is every year

##### Wind

In [9]:
# Auto-correlation for wind
wind_autocorrelation_lag1 = combined['Actual Wind (MW)'].autocorr(lag=1)
wind_autocorrelation_lag3 = combined['Actual Wind (MW)'].autocorr(lag=3)
wind_autocorrelation_lag6 = combined['Actual Wind (MW)'].autocorr(lag=6)
# wind_autocorrelation_lag9 = combined['Actual Wind (MW)'].autocorr(lag=9)
wind_autocorrelation_lag12 = combined['Actual Wind (MW)'].autocorr(lag=12)

print(" 1 Month Lag for wind: ", wind_autocorrelation_lag1)
print(" 3 Month Lag for wind: ", wind_autocorrelation_lag3)
print(" 6 Month Lag for wind: ", wind_autocorrelation_lag6)
# print(" 9 Month Lag for wind: ", wind_autocorrelation_lag9)
print("12 Month Lag for wind: ", wind_autocorrelation_lag12)


##### Demand

In [10]:
# Auto-correlation for demand
demand_autocorrelation_lag1 = combined['Demand (MW)'].autocorr(lag=1)
demand_autocorrelation_lag3 = combined['Demand (MW)'].autocorr(lag=3)
demand_autocorrelation_lag6 = combined['Demand (MW)'].autocorr(lag=6)
demand_autocorrelation_lag12 = combined['Demand (MW)'].autocorr(lag=12)

print(" 1 Month Lag for demand: ", demand_autocorrelation_lag1)
print(" 3 Month Lag for demand: ", demand_autocorrelation_lag3)
print(" 6 Month Lag for demand: ", demand_autocorrelation_lag6)
print("12 Month Lag for demand: ", demand_autocorrelation_lag12)


##### Net Demand

In [11]:
# Auto-correlation for net demand
net_demand_autocorrelation_lag1 = combined['Net Demand'].autocorr(lag=1)
net_demand_autocorrelation_lag3 = combined['Net Demand'].autocorr(lag=3)
net_demand_autocorrelation_lag6 = combined['Net Demand'].autocorr(lag=6)
net_demand_autocorrelation_lag12 = combined['Net Demand'].autocorr(lag=12)

print(" 1 Month Lag for net demand: ", net_demand_autocorrelation_lag1)
print(" 3 Month Lag for net demand: ", net_demand_autocorrelation_lag3)
print(" 6 Month Lag for net demand: ", net_demand_autocorrelation_lag6)
print("12 Month Lag for net demand: ", net_demand_autocorrelation_lag12)

##### Gas Price

In [12]:
# Auto-correlation for gas price
gas_price_autocorrelation_lag1 = combined['Gas Price £/Therm'].autocorr(lag=1)
gas_price_autocorrelation_lag3 = combined['Gas Price £/Therm'].autocorr(lag=3)
gas_price_autocorrelation_lag6 = combined['Gas Price £/Therm'].autocorr(lag=6)
gas_price_autocorrelation_lag12 = combined['Gas Price £/Therm'].autocorr(lag=12)

print(" 1 Month Lag for gas price: ", gas_price_autocorrelation_lag1)
print(" 3 Month Lag for gas price: ", gas_price_autocorrelation_lag3)
print(" 6 Month Lag for gas price: ", gas_price_autocorrelation_lag6)
print("12 Month Lag for gas price: ", gas_price_autocorrelation_lag12)

##### GB Electricity Prices

In [13]:
# Auto-correlation for electricity price
elec_price_autocorrelation_lag1 = combined['GB Price (€/MWh)'].autocorr(lag=1)
elec_price_autocorrelation_lag3 = combined['GB Price (€/MWh)'].autocorr(lag=3)
elec_price_autocorrelation_lag6 = combined['GB Price (€/MWh)'].autocorr(lag=6)
elec_price_autocorrelation_lag12 = combined['GB Price (€/MWh)'].autocorr(lag=12)

print(" 1 Month Lag for electricity price: ", elec_price_autocorrelation_lag1)
print(" 3 Month Lag for electricity price: ", elec_price_autocorrelation_lag3)
print(" 6 Month Lag for electricity price: ", elec_price_autocorrelation_lag6)
print("12 Month Lag for electricity price: ", elec_price_autocorrelation_lag12)

##### Prices - DAM

In [14]:
# Auto-correlation for DAM market
dam_autocorrelation_lag1 = combined['DAM Market Price (€/MWh)'].astype('float32').autocorr(lag=1)
dam_autocorrelation_lag3 = combined['DAM Market Price (€/MWh)'].astype('float32').autocorr(lag=3)
dam_autocorrelation_lag6 = combined['DAM Market Price (€/MWh)'].astype('float32').autocorr(lag=6)
dam_autocorrelation_lag12 = combined['DAM Market Price (€/MWh)'].astype('float32').autocorr(lag=12)

print(" 1 Month Lag for DAM market price: ", dam_autocorrelation_lag1)
print(" 3 Month Lag for DAM market price: ", dam_autocorrelation_lag3)
print(" 6 Month Lag for DAM market price: ", dam_autocorrelation_lag6)
print("12 Month Lag for DAM market price: ", dam_autocorrelation_lag12)

##### Prices - IDA1

In [15]:
# Auto-correlation for IDA1 market
ida1_autocorrelation_lag1 = combined['IDA1 Market Price (€/MWh)'].astype('float32').autocorr(lag=1)
ida1_autocorrelation_lag3 = combined['IDA1 Market Price (€/MWh)'].astype('float32').autocorr(lag=3)
ida1_autocorrelation_lag6 = combined['IDA1 Market Price (€/MWh)'].astype('float32').autocorr(lag=6)
ida1_autocorrelation_lag12 = combined['IDA1 Market Price (€/MWh)'].astype('float32').autocorr(lag=12)

print(" 1 Month Lag for IDA1 market price: ", ida1_autocorrelation_lag1)
print(" 3 Month Lag for IDA1 market price: ", ida1_autocorrelation_lag3)
print(" 6 Month Lag for IDA1 market price: ", ida1_autocorrelation_lag6)
print("12 Month Lag for IDA1 market price: ", ida1_autocorrelation_lag12)

##### Prices - IDA2

In [16]:
# Auto-correlation for IDA2 market
ida2_autocorrelation_lag1 = combined['IDA2 Market Price (€/MWh)'].astype('float32').autocorr(lag=1)
ida2_autocorrelation_lag3 = combined['IDA2 Market Price (€/MWh)'].astype('float32').autocorr(lag=3)
ida2_autocorrelation_lag6 = combined['IDA2 Market Price (€/MWh)'].astype('float32').autocorr(lag=6)
ida2_autocorrelation_lag12 = combined['IDA2 Market Price (€/MWh)'].astype('float32').autocorr(lag=12)

print(" 1 Month Lag for IDA2 market price: ", ida2_autocorrelation_lag1)
print(" 3 Month Lag for IDA2 market price: ", ida2_autocorrelation_lag3)
print(" 6 Month Lag for IDA2 market price: ", ida2_autocorrelation_lag6)
print("12 Month Lag for IDA2 market price: ", ida2_autocorrelation_lag12)

##### Prices - IDA3

In [17]:
# Auto-correlation for IDA3 market
ida3_autocorrelation_lag1 = combined['IDA3 Market Price (€/MWh)'].astype('float32').autocorr(lag=1)
ida3_autocorrelation_lag3 = combined['IDA3 Market Price (€/MWh)'].astype('float32').autocorr(lag=3)
ida3_autocorrelation_lag6 = combined['IDA3 Market Price (€/MWh)'].astype('float32').autocorr(lag=6)
ida3_autocorrelation_lag12 = combined['IDA3 Market Price (€/MWh)'].astype('float32').autocorr(lag=12)

print(" 1 Month Lag for IDA3 market price: ", ida3_autocorrelation_lag1)
print(" 3 Month Lag for IDA3 market price: ", ida3_autocorrelation_lag3)
print(" 6 Month Lag for IDA3 market price: ", ida3_autocorrelation_lag6)
print("12 Month Lag for IDA3 market price: ", ida3_autocorrelation_lag12)

##### Prices - BM

In [18]:
# Auto-correlation for BM market
bm_autocorrelation_lag1 = combined['BM Market Price (€/MWh)'].astype('float32').autocorr(lag=1)
bm_autocorrelation_lag3 = combined['BM Market Price (€/MWh)'].astype('float32').autocorr(lag=3)
bm_autocorrelation_lag6 = combined['BM Market Price (€/MWh)'].astype('float32').autocorr(lag=6)
bm_autocorrelation_lag12 = combined['BM Market Price (€/MWh)'].astype('float32').autocorr(lag=12)

print(" 1 Month Lag for BM market price: ", bm_autocorrelation_lag1)
print(" 3 Month Lag for BM market price: ", bm_autocorrelation_lag3)
print(" 6 Month Lag for BM market price: ", bm_autocorrelation_lag6)
print("12 Month Lag for BM market price: ", bm_autocorrelation_lag12)

### Stationarity

For stationality, we use Dickey Fuller test to help. This test will apply statistic hypothesis on the value, if the alternative hypothesis is rejected, then the data is stationary.  (Pierre, 2022)

##### GB Electricity Prices

In [41]:
#Electricity

#Create copy to not affect original dataset
elec_price_stationary = demand_electricity.copy()

#Need to reset this to enable conversion 
elec_price_stationary["Start Date"] = elec_price_stationary["Start Date"].astype(str)

#Make date time column
elec_price_stationary['datetime'] = pd.to_datetime(elec_price_stationary['Start Date'] + ' ' + elec_price_stationary['Start Time 30 Minute Period'])

#Keep only the columns of interest
elec_price_stationary= elec_price_stationary.drop(["Start Time 30 Minute Period", "Start Date", "Trade Date"], axis = 1)

#set datetime as index
elec_price_stationary.set_index(['datetime'], inplace=True)

In [19]:
# The more negative the ADF, the more likely we are to reject the null
# Null Hypothesis: Suggests the time series has a unit root, meaning it is non-stationary; it has some time dependent structure.
# Alternative Hypothesis: Suggests the time series is stationary. It does not have time-dependent structure.

#p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
#p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.


from statsmodels.tsa.stattools import adfuller
print("Observations of Dickey-fuller test")
result = adfuller(elec_price_stationary['GB Price (€/MWh)'],autolag='AIC')

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

##### Gas Price

In [86]:
# Gas

#Create copy to not affect original dataset
gas_price_stationary = demand_gas_half_hour.copy()

#Need to reset this to enable conversion 
gas_price_stationary["Start Date"] = gas_price_stationary["Start Date"].astype(str)

#Make date time column
gas_price_stationary['datetime'] = pd.to_datetime(gas_price_stationary['Start Date'] + ' ' + gas_price_stationary['Start Time 30 Minute Period'])

#Keep only the columns of interest
gas_price_stationary= gas_price_stationary.drop(["Start Time 30 Minute Period", "Start Date", "Trade Date"], axis = 1)

#set datetime as index
gas_price_stationary.set_index(['datetime'], inplace=True)


In [20]:
# The more negative the ADF, the more likely we are to reject the null
# Null Hypothesis: Suggests the time series has a unit root, meaning it is non-stationary; it has some time dependent structure.
# Alternative Hypothesis: Suggests the time series is stationary. It does not have time-dependent structure.

#p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
#p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

from statsmodels.tsa.stattools import adfuller
print("Observations of Dickey-fuller test")
result = adfuller(gas_price_stationary['Gas Price £/Therm'],autolag='AIC')

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

##### Wind

In [88]:
# Wind

#Create copy to not affect original dataset
wind_stationary = combined.copy()

wind_stationary["Start Date"] = wind_stationary["Start Date"].astype(str)

#Make date time column
wind_stationary['datetime'] = pd.to_datetime(wind_stationary['Start Date'] + ' ' + wind_stationary['Start Time 30 Minute Period'])

#Keep only the columns of interest
wind_stationary= wind_stationary.drop(["Start Time 30 Minute Period", "Start Date", "Trade Date", 'Net Demand', 'Demand (MW)'], axis = 1)

#set datetime as index
wind_stationary.set_index(['datetime'], inplace=True)


In [21]:
# The more negative the ADF, the more likely we are to reject the null
# Null Hypothesis: Suggests the time series has a unit root, meaning it is non-stationary; it has some time dependent structure.
# Alternative Hypothesis: Suggests the time series is stationary. It does not have time-dependent structure.

#p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
#p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.


from statsmodels.tsa.stattools import adfuller
print("Observations of Dickey-fuller test")
result = adfuller(wind_stationary['Actual Wind (MW)'],autolag='AIC')

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

##### Demand

In [90]:
# Demand

#Create copy to not affect original dataset
demand_stationary = combined.copy()

demand_stationary["Start Date"] = demand_stationary["Start Date"].astype(str)

#Make date time column
demand_stationary['datetime'] = pd.to_datetime(demand_stationary['Start Date'] + ' ' + demand_stationary['Start Time 30 Minute Period'])

#Keep only the columns of interest
demand_stationary = demand_stationary.drop(["Start Time 30 Minute Period", "Start Date", "Trade Date", 'Net Demand', 'Actual Wind (MW)'], axis = 1)

#set datetime as index
demand_stationary.set_index(['datetime'], inplace=True)


In [22]:
# The more negative the ADF, the more likely we are to reject the null
# Null Hypothesis: Suggests the time series has a unit root, meaning it is non-stationary; it has some time dependent structure.
# Alternative Hypothesis: Suggests the time series is stationary. It does not have time-dependent structure.

#p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
#p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.


from statsmodels.tsa.stattools import adfuller
print("Observations of Dickey-fuller test")
result = adfuller(demand_stationary['Demand (MW)'],autolag='AIC')

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

### Decomposition

By spliting a time series data into several components, each representing an underlying pattern category.We can see the decomposition plot comprising three components: a trend-cycle component, a seasonal component, and a remainder component (containing anything else in the time series)().

##### Wind

In [23]:
# Decomposition for wind
# The "period" parameter is the number of observations in a seasonal cycle. 
# For example, if you have daily observations and weekly seasonality, the period is 7. 
# Hence period here is 365 days a year * 48 time slots a day = 17520 
 
wind_decompose = seasonal_decompose(combined['Actual Wind (MW)'],model='additive', period=17520)
wind_decompose.plot()
plt.show()

##### Demand

In [24]:
demand_decompose = seasonal_decompose(combined['Demand (MW)'],model='additive', period=17520)
demand_decompose.plot()
plt.show()

##### Net Demand

In [25]:
net_demand_decompose = seasonal_decompose(combined['Net Demand'],model='additive', period=17520)
net_demand_decompose.plot()
plt.show()

##### Gas Price

In [26]:
gas_decompose = seasonal_decompose(combined['Gas Price £/Therm'],model='additive', period=17520)
gas_decompose.plot()
plt.show()

##### Electricity Prices

In [27]:
gb_price_decompose = seasonal_decompose(combined['GB Price (€/MWh)'],model='additive', period=17520)
gb_price_decompose.plot()
plt.show()

# Modelling

To handle time sequence data, LSTM is a good choice. LSTM models are a type of recurrent neural network that are designed to capture these temporal dependencies in time series data. The unique architecture of LSTM models allows them to selectively remember and forget information from previous time steps, making them well-suited for processing long sequences of data.  It can effectively capture dependencies and patterns that exist over long time lags in the data. 

The input data was split into train, validation and test. The test data contains the last 48 time slots of a day to evaluate the predictions. We splited the rest of the data into 80/20, which train data accounts for 80% and validation accounts for 20%. 

Initially, we decided to use a single model to predict prices of all markets together, however due to the different opening times of IDA2 and IDA3 market, the model results in extremely high loss during training and bad predictions. Therefore, we determined to build 3 models - one for DAM, IDA1, BM since they are all open 24 hours, one for IDA2 and another for IDA3. For the closed period of IDA2 and IDA3, we decided to fill the NAN with the median values of that day after several attempts. 

The training loss of each model is shown as follows. These curves decrease as the training process proceeds and then reaches a stable point. After this, we apply the model to the test dataset and creates a list of prediction. By calculating the Root Mean Squared Error(RMSE) between the prediction and our true values, we will know the performance of our model on test set, and the smaller the RMSE is, the closer the prediction is to the real data, the better result we have. The average RMSE of these three tests are within the range 40-50, which means the model works but still could be improved. This is thelimitation of out model and we will discuss this in the later part.

After finishing testing the data of the last day, we append our predictions to the “combined” dataframe which saves all our variables, and the columns name add “prediction” at the start. So now we have both predictions and true values in the same data frame and we can compare them easily. 

In [44]:
# import libraries for models

from numpy import array
from numpy import hstack
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [224]:
# Interpolate 0 values in IDA2 and IDA3 before model training

combined_bkp = combined.copy()
df= combined.copy()

#Group by Start Date
df_grouped = df.groupby(df['Start Date'].dt.date)

# Calculate the mean of non-zero values for date in ID2
median_ida2 = df_grouped['IDA2 Market Price (€/MWh)'].transform(lambda x: x[x != 0].median())
median_ida3 = df_grouped['IDA3 Market Price (€/MWh)'].transform(lambda x: x[x != 0].median())

#In case of error; unlikely a few 0s will affcect prediction that badly
median_ida2.fillna(0, inplace = True)
median_ida3.fillna(0, inplace = True)

# Fill the zeros with the mean value for each day
combined['IDA2 Market Price (€/MWh)'] = df["IDA2 Market Price (€/MWh)"].mask(df['IDA2 Market Price (€/MWh)'] == 0, median_ida2)
combined['IDA3 Market Price (€/MWh)'] = df["IDA3 Market Price (€/MWh)"].mask(df['IDA3 Market Price (€/MWh)'] == 0, median_ida3)

In [225]:
# Define input sequences

# get column values for reshaping
wind_input = combined["Actual Wind (MW)"]
demand_input = combined["Demand (MW)"]
net_demand_input = combined["Net Demand"]
gas_input = combined["Gas Price £/Therm"]
elec_input = combined["GB Price (€/MWh)"]
dm_output = combined["DAM Market Price (€/MWh)"]
ida1_output = combined["IDA1 Market Price (€/MWh)"]
ida2_output = combined["IDA2 Market Price (€/MWh)"]
ida3_output = combined["IDA3 Market Price (€/MWh)"]
bm_output = combined['BM Market Price (€/MWh)']

# reshape
wind_input = wind_input.values.reshape((-1, 1))
demand_input = demand_input.values.reshape((-1, 1))
net_demand_input = net_demand_input.values.reshape((-1, 1))
gas_input = gas_input.values.reshape((-1, 1))
elec_input = elec_input.values.reshape((-1, 1))
dm_output = dm_output.values.reshape((-1,1))
ida1_output = ida1_output.values.reshape((-1,1))
ida2_output = ida2_output.values.reshape((-1,1))
ida3_output = ida3_output.values.reshape((-1,1))
bm_output = bm_output.values.reshape((-1,1))


# horizontally stack columns

# for markets DAM, ID1 and BM
dataset = hstack((wind_input, demand_input, net_demand_input, gas_input, elec_input, dm_output, ida1_output, bm_output))
# for IDA2 market
dataset_ida2 = hstack((wind_input, demand_input, net_demand_input, gas_input, elec_input, ida2_output))
# for IDA3 market
dataset_ida3 = hstack((wind_input, demand_input, net_demand_input, gas_input, elec_input, ida3_output))

# create train, validation and test datasets for all three cases
test_data_3M = dataset[-53:] # add extra rows to get 48 rows at the end as this is also done in batches
train_data_3M = dataset[0:31488]
val_data_3M = dataset[31488:-53]

test_data_ida2 = dataset_ida2[-53:] ############################
train_data_ida2 = dataset_ida2[0:31488]
val_data_ida2 = dataset_ida2[31488:-53]

test_data_ida3 = dataset_ida3[-53:]
train_data_ida3 = dataset_ida3[0:31488]
val_data_ida3 = dataset_ida3[31488:-53]


# print(dataset[0])
# print("--------")
# print(dataset_ida2[0])
# print("--------")
# print(dataset_ida3[0])

In [28]:
# ensure split is proper
len(val_data_3M) + len(train_data_3M) + len(test_data_3M)
len(val_data_ida2) + len(train_data_ida2) + len(test_data_ida2)
len(val_data_ida3) + len(train_data_ida3) + len(test_data_ida3)

In [227]:
# test_data_3M[-1,:-3]
# test_data_3M[-1,-3:]
# # [1250.      , 4498.      , 3248.      ,  114.3     ,  118.1856  ,         121.4     ,  110.      ,  112.88    ,  112.88    ,  170.94    ]
# print(" ")
# test_data_ida2[-1,:-1]
# test_data_ida2[-1,-1]

In [228]:
### define functions to split multivariate sequence into samples

# For DAM, IDA1 and BM Markets
def split_sequences_3M(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        # seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        seq_x, seq_y = sequences[i:end_ix, :-3], sequences[end_ix-1, -3:]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


# For IDA2 and IDA3 markets
def split_sequences_1M(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# If we have 8 rows(A-H) altogether and our step is 4, it means we will first make ABCD as input and E as output. Then we will choose BCDE as input and F as output.

In [229]:
# define number of steps
n_steps = 4

# convert into input/output
# For DAM, IDA1 and BM markets
x_train_3M, y_train_3M = split_sequences_3M(train_data_3M, n_steps)
# For IDA2 market
x_train_ida2, y_train_ida2 = split_sequences_1M(train_data_ida2, n_steps)
# For IDA3 market
x_train_ida3, y_train_ida3 = split_sequences_1M(train_data_ida3, n_steps)

# print(x_train_3M.shape, y_train_3M.shape)
# print(x_train_3M[0], y_train_3M[0])
# print("--------")
# print(x_train_ida2.shape, y_train_ida2.shape)
# print(x_train_ida2[0], y_train_ida2[0])
# print("--------")
# print(x_train_ida3.shape, y_train_ida3.shape)
# print(x_train_ida3[0], y_train_ida3[0])

In [29]:
#--------------------------------------------------------------------------------------------------------------
# Model for DAM, IDA1 and BM Markets
#--------------------------------------------------------------------------------------------------------------

# define model
model_3M = Sequential()
model_3M.add(LSTM(100,  input_shape=(n_steps, x_train_3M.shape[2])))
model_3M.add(Dense(3))

# compile model
model_3M.compile(optimizer='adam', loss='mse')

# fit model
x_train_3M = np.asarray(x_train_3M).astype(np.float32)
y_train_3M = np.asarray(y_train_3M).astype(np.float32)

# Instantiate the model and save the loss
history_3M = model_3M.fit(x_train_3M, y_train_3M, epochs=60, verbose=1)


In [30]:
#--------------------------------------------------------------------------------------------------------------
# Model for IDA2 market
#--------------------------------------------------------------------------------------------------------------

# define model
model_ida2 = Sequential()
model_ida2.add(LSTM(100,  input_shape=(n_steps, x_train_ida2.shape[2])))
model_ida2.add(Dense(1))

# Compile model
model_ida2.compile(optimizer='adam', loss='mse')

# fit model
x_train_ida2 = np.asarray(x_train_ida2).astype(np.float32)
y_train_ida2 = np.asarray(y_train_ida2).astype(np.float32)

# Instantiate the model and save the loss
history_ida2 = model_ida2.fit(x_train_ida2, y_train_ida2, epochs=60, verbose=1)


In [31]:
#--------------------------------------------------------------------------------------------------------------
# Model for IDA3 market
#--------------------------------------------------------------------------------------------------------------

# define model
model_ida3 = Sequential()
model_ida3.add(LSTM(100,  input_shape=(n_steps, x_train_ida3.shape[2])))
# model_ida3.add(LSTM(100))
model_ida3.add(Dense(1))

# Compile model
model_ida3.compile(optimizer='adam', loss='mse')

# fit model
x_train_ida3 = np.asarray(x_train_ida3).astype(np.float32)
y_train_ida3 = np.asarray(y_train_ida3).astype(np.float32)

# Instantiate the model and save the loss
history_ida3 = model_ida3.fit(x_train_ida3, y_train_ida3, epochs=60, verbose=1)


In [32]:
# Save loss outputs to enable plotting
hist_df_3M = pd.DataFrame(history_3M.history) 
hist_df_ida2 = pd.DataFrame(history_ida2.history) 
hist_df_ida3 = pd.DataFrame(history_ida3.history) 

plt.plot(history_3M.history['loss'], label='Training loss DAM, IDA1, BM')
plt.plot(history_ida2.history['loss'], label='Training loss IDA2')
plt.plot(history_ida3.history['loss'], label='Training loss IDA3')
plt.legend();

In [234]:
# create sequences for validation

x_val_3M, y_val_3M = split_sequences_3M(val_data_3M, n_steps)
x_val_ida2, y_val_ida2 = split_sequences_1M(val_data_ida2, n_steps)
x_val_ida3, y_val_ida3 = split_sequences_1M(val_data_ida3, n_steps)

# print(x_val_3M.shape, y_val_3M.shape)
# print(x_val_ida2.shape, y_val_ida2.shape)
# print(x_val_ida3.shape, y_val_ida3.shape)

# print(x_val_3M[0], y_val_3M[0])
# print(x_val_ida2[0], y_val_ida2[0])
# print(x_val_ida3[0], y_val_ida3[0])


In [33]:
# Predict for DAM, ID1, BM Markets
x_val_3M = np.asarray(x_val_3M).astype(np.float32)
y_val_3M = np.asarray(y_val_3M).astype(np.float32)
prediction_3M = model_3M.predict(x_val_3M, verbose=1)
# print(prediction_3M)

In [34]:
# Predict for IDA2 market
x_val_ida2 = np.asarray(x_val_ida2).astype(np.float32)
y_val_ida2 = np.asarray(y_val_ida2).astype(np.float32)
prediction_ida2 = model_ida2.predict(x_val_ida2, verbose=1)
# print(prediction_ida2)

In [35]:
# Predict for IDA3 market
x_val_ida3 = np.asarray(x_val_ida3).astype(np.float32)
y_val_ida3 = np.asarray(y_val_ida3).astype(np.float32)
prediction_ida3 = model_ida3.predict(x_val_ida3, verbose=1)
# print(prediction_ida3)

In [36]:
# Check root mean square error on validation data

rmse_3M = []
rmse_ida2 = []
rmse_ida3 = []

for i in range(len(y_val_3M)):
    rmse_3M.append(np.sqrt(np.mean((prediction_3M[i] - y_val_3M[i])**2)))
    
for i in range(len(y_val_ida2)):
    rmse_ida2.append(np.sqrt(np.mean((prediction_ida2[i] - y_val_ida2[i])**2)))

for i in range(len(y_val_ida3)):
    rmse_ida3.append(np.sqrt(np.mean((prediction_ida3[i] - y_val_ida3[i])**2)))

# Find minimum, average and maximum RMSE values
print("Statistics for DAM, IDA1 and BM markets")
print("----------------------------------------")
print(f"max: {max(rmse_3M)}, min:{min(rmse_3M)}, avg: {np.mean(rmse_3M)}")

print("IDA2 market")
print("----------------------------------------")
print(f"max: {max(rmse_ida2)}, min:{min(rmse_ida2)}, avg: {np.mean(rmse_ida2)}")

print("IDA3 market")
print("----------------------------------------")
print(f"max: {max(rmse_ida3)}, min:{min(rmse_ida3)}, avg: {np.mean(rmse_ida3)}")

In [37]:
# Prove RMSE decreases

# Create subplots
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(12,4))

# Plot the histograms on each subplot
ax1.hist(rmse_3M, bins=30, color='#45ADA8')
ax2.hist(rmse_ida2, bins=30, color='#9DE0AD')
ax3.hist(rmse_ida3, bins=30, color='#FFA500')

# Set the axis labels and titles for each subplot
ax1.set_xlabel('RMSE')
ax1.set_ylabel('Probability')
ax1.set_title('Histogram of rmse_3M')

ax2.set_xlabel('RMSE')
ax2.set_ylabel('Probability')
ax2.set_title('Histogram of rmse_ida2')

ax3.set_xlabel('RMSE')
ax3.set_ylabel('Probability')
ax3.set_title('Histogram of rmse_ida3')

# Show the plot
plt.show();

In [240]:
# using test dataset to check

test_data_X_3M = test_data_3M[:, :-3] # all columns except last n
test_data_Y_3M = test_data_3M[:, -3:] # last n columns

test_data_X_ida2 = test_data_ida2[:, :-1] 
test_data_Y_ida2 = test_data_ida2[:, -1:] 

test_data_X_ida3 = test_data_ida3[:, :-1] 
test_data_Y_ida3 = test_data_ida3[:, -1:] 

# print("--------")
# test_data_X_3M[3]
# test_data_Y_3M[3]
# print("--------")
# test_data_X_ida2[3]
# test_data_Y_ida2[3]
# print("--------")
# test_data_X_ida3[3]
# test_data_Y_ida3[3]

In [241]:
# Split into sequence
n_steps = 4

test_data_X_3M, test_data_Y_3M = split_sequences_3M(test_data_3M, n_steps)
test_data_X_3M, test_data_Y_3M = split_sequences_3M(test_data_3M, n_steps)

test_data_X_ida2, test_data_Y_ida2 = split_sequences_1M(test_data_ida2, n_steps)
test_data_X_ida2, test_data_Y_ida2 = split_sequences_1M(test_data_ida2, n_steps)

test_data_X_ida3, test_data_Y_ida3 = split_sequences_1M(test_data_ida3, n_steps)
test_data_X_ida3, test_data_Y_ida3 = split_sequences_1M(test_data_ida3, n_steps)

In [242]:
# Predict for DAM, IDA1 and BM markets on test data
test_data_X_3M = np.asarray(test_data_X_3M).astype(np.float32)
test_data_Y_3M = np.asarray(test_data_Y_3M).astype(np.float32)
prediction_test_3M = model_3M.predict(test_data_X_3M, verbose=0)
# print(prediction_test_3M) 

In [243]:
# Predict for IDA2 market on test data
test_data_X_ida2 = np.asarray(test_data_X_ida2).astype(np.float32)
test_data_Y_ida2 = np.asarray(test_data_Y_ida2).astype(np.float32)
prediction_test_ida2 = model_ida2.predict(test_data_X_ida2, verbose=0)
# print(prediction_test_ida2) 

In [244]:
# Predict for IDA3 market on test data
test_data_X_ida3 = np.asarray(test_data_X_ida3).astype(np.float32)
test_data_Y_ida3 = np.asarray(test_data_Y_ida3).astype(np.float32)
prediction_test_ida3 = model_ida3.predict(test_data_X_ida3, verbose=0)
# print(prediction_test_ida3)

## Results and Discussion
We apply the model on 31 March 2023’s data and measure its performance. From the plots that we can observed that the curve stays still until 11am, at which point it starts to increase and starts to follow the real trend. The model first selected IDA1 until 8am, and then BM until 11am, after that we choose IDA2. This result shows that the model under estimate the cost of IDA1 energy prices during this time, whilst overestimating DAM and BM. This illustrates a drawback of a basic LSTM model, which relies subsequent decisions in the time series on memory from prior data. Therefore, the best plan of action would be to only help purchasing selections made after 11 a.m. 

In [250]:
# Show test data with predictions

# copy last 48 rows (last 1 day data)
combined_test = combined[-50:].copy() ###########################

# create a combined dataframe with predictions for all markets
predictions_DAM = pd.DataFrame(prediction_test_3M[:, 0], columns=['Predicted DAM Market Price'])
predictions_IDA1 = pd.DataFrame(prediction_test_3M[:, 1], columns=['Predicted IDA1 Price'])
predictions_IDA2 = pd.DataFrame(prediction_test_ida2[:, 0], columns=['Predicted IDA2 Price'])
predictions_IDA3 = pd.DataFrame(prediction_test_ida3[:, 0], columns=['Predicted IDA3 Price'])
predictions_BM = pd.DataFrame(prediction_test_3M[:, 2], columns=['Predicted BM Price'])

combined_test['Predicted DAM Market Price'] = predictions_DAM['Predicted DAM Market Price'].values
combined_test['Predicted IDA1 Price'] = predictions_IDA1['Predicted IDA1 Price'].values
combined_test['Predicted IDA2 Price'] = predictions_IDA2['Predicted IDA2 Price'].values
combined_test['Predicted IDA3 Price'] = predictions_IDA3['Predicted IDA3 Price'].values
combined_test['Predicted BM Price'] = predictions_BM['Predicted BM Price'].values

combined_test.drop(combined_test.tail(1).index,inplace=True) # pop off last row

In [40]:
# copy original market values into another dataframe

original_values = combined_test[["Start Date", "Trade Date", "Start Time 30 Minute Period",
                                 "DAM Market Price (€/MWh)",
                                 "IDA1 Market Price (€/MWh)",
                                 "IDA2 Market Price (€/MWh)",
                                 "IDA3 Market Price (€/MWh)",
                                 "BM Market Price (€/MWh)",  
                                 ]].copy()

original_values.head(1)

In [41]:
# Gather market price predictions 

predicted_values = combined_test.drop(["Gas Price £/Therm", "GB Price (€/MWh)",
                    "Demand (MW)", "Actual Wind (MW)", "Net Demand",
                    "Trade Date", "Start Date",
                     "Start Time 30 Minute Period",
                     "DAM Market Price (€/MWh)",
                     "IDA1 Market Price (€/MWh)",
                     "IDA2 Market Price (€/MWh)",
                     "IDA3 Market Price (€/MWh)",
                     "BM Market Price (€/MWh)",
                     "Market Net Imbalance Volume (MW)"], axis = 1)

predicted_values.head(1)

In [254]:
# Create 30 minute intervals to append later
time_intervals = combined_test[["Start Time 30 Minute Period"]].copy()

In [255]:
# Function to identify most profitable market

def profit_detector (df):
    
    
    highest_col = []
    # Create list to hold the highest profit value for each row
    highest_vals = []
    markets = ['DAM', 'IDA1', 'IDA2', 'IDA3', 'BM']
    

    for i, row in df.iterrows():
        
        # Create a list to hold the proft values calculated for each row after applying the equation
        row_vals = []
        index_loc= []


        # Iterate through columns and calculate the value based on Energia's profitability formula
        for j in range(0, 5): 
    
            # as IDA2 And 3 have 0 values, we need to pass over these
            if row[j] == 0:
                pass
            
            elif row[j] > row[0]:
                pass
            
            elif row[j] == row[0]:
                # net profitability is zero when the values are same
                row_vals.append(((row[0] - row[j]) * 100) / 2)
                # also save corresponding index location 
                index_loc.append(j)
                
            elif row[j] < row[0]:
                row_vals.append(((row[0] - row[j]) * 100) / 2)
                # also save corresponding index location 
                index_loc.append(j)
                
            
        # Find the index of the column that contains the highest profitability value contained in row_vals and save as var
        max_col_index = row_vals.index(max(row_vals))

        # Add the most profitable value and its corresponding column name to the respective list
        highest_vals.append(max(row_vals))
        highest_col.append(markets[index_loc[max_col_index]])
        
    # Rename the lists to make sense in the context of the data
    df["Max predicted profitability (€/MWh)"] = highest_vals
    df["Most Profitable Predicted market"] = highest_col
    
    # Return the data frame with the new columns
    return df

In [256]:
# Apply a mask to the values that should be 0 in IDA2 & 3
# They aren't 0 as they we're filled with the median 

arge = pd.concat([time_intervals, predicted_values], axis=1)
arge['Predicted IDA2 Price'].mask((arge['Start Time 30 Minute Period'] <"11:00:00") | (arge['Start Time 30 Minute Period'] >"17:00:00"),'0', inplace=True)
arge['Predicted IDA3 Price'].mask((arge['Start Time 30 Minute Period'] <"17:00:00") | (arge['Start Time 30 Minute Period'] >"23:00:00"),'0', inplace=True)
arge.drop(["Start Time 30 Minute Period"], inplace = True, axis = 1)
arge = arge.astype(float)

In [257]:
# Get dataframe with profit details 
df = profit_detector(arge)
result = pd.concat([original_values, df], axis=1)

In [38]:
# Show actual values and predictions side by side
# result = (result[['Start Date', 'Trade Date', 'Start Time 30 Minute Period',
#          'DAM Market Price (€/MWh)', 'Predicted DAM Market Price',
#         'IDA1 Market Price (€/MWh)', 'Predicted IDA1 Price',
#          'IDA2 Market Price (€/MWh)', 'Predicted IDA2 Price',
#          'IDA3 Market Price (€/MWh)', 'Predicted IDA3 Price',
#          'BM Market Price (€/MWh)', 'Predicted BM Price',
#         "Max predicted profitability (€/MWh)", "Profitable market" 
#         ]]).copy()

# predict for last day (48 intervals)
result

In [259]:
# Make a DF of: DAMN, Actual DAM Minus Pred Market, Actual DAM Minus Actual Markets, Pred DAM Minus Pred Markets



In [266]:
# Gets DAMN and pred values
true_dam = original_values["DAM Market Price (€/MWh)"]
pred = predicted_values.drop("Predicted DAM Market Price", axis = 1).copy()
testy = pred.assign(DAM = true_dam)
testy = testy[['DAM', 'Predicted IDA1 Price', 'Predicted IDA2 Price', 'Predicted IDA3 Price', "Predicted BM Price"]].copy()
dam_on_pred = profit_detector (testy)


In [267]:
# Gets DAMN and actual
true_vals = original_values.drop(["Start Date", "Trade Date", "Start Time 30 Minute Period"], axis =1).copy()
damn_on_true = profit_detector (true_vals)

In [300]:
# Make a DF of: DAMN, Actual DAM Minus Pred Market, Actual DAM Minus Actual Markets, Pred DAM Minus Pred Markets

#Pred values
a = result["Max predicted profitability (€/MWh)"].copy()
a = pd.DataFrame(a)

#DAM
a['DAM'] = 0 

#Start (x)
a["Start Time 30 Minute Period"] = time_intervals # Pop in time intervals

#Pred
a["Actual DAM Minus Pred"] = dam_on_pred["Max predicted profitability (€/MWh)"]

#True
a["Actual Max Profit"] = damn_on_true["Max predicted profitability (€/MWh)"]

In [320]:
a.drop("Actual DAM Minus Pred", axis = 1, inplace = True)

In [42]:
#a=a.set_index('Start Time 30 Minute Period').copy()

plt = a.plot.line()


In [39]:
combined.tail(60)

## Conclusion & Recommendation

In conclusion, the model has been successfully created to help predict market prices according to the given variables, however, for this project, it still doesn't adequately address Energia's business issues with the necessary level of realism. The shortage of time and complexity of the project make us not have adequate time on this project, therefore, to optimize the model and give better predictions, we give the following recommendations: 
1. Try to apply normalization to the data as pre-processing, since it will lead to significantly better results than the unnormalized data. (Hou et al., 2019)
2. We only focus on building a model that could give predictions rather than a perfect model, hence we can apply optimization to the model to improve the performance. 


## Reference
1. Bonthu, H. (2023) Detecting and treating outliers: Treating the odd one out!, Analytics Vidhya. Available at: https://www.analyticsvidhya.com/blog/2021/05/detecting-and-treating-outliers-treating-the-odd-one-out/ (Accessed: May 7, 2023). 
2. Brownlee, J. (2019) Time series data visualization with python, MachineLearningMastery.com. Available at: https://machinelearningmastery.com/time-series-data-visualization-with-python/ (Accessed: May 7, 2023). 
3. Forecasting: Principles and practice (2nd ed) (no date) Chapter 6 Time series decomposition. Available at: https://otexts.com/fpp2/decomposition.html (Accessed: May 7, 2023). 
4. Hou, L., Zhu, J., T. Kwok, J., Gao, F., Qin, T. and Liu, T. (2019). Normalization Helps Training of Quantized LSTM. Advances in Neural Information Processing Systems 32 (NeurIPS 2019). [online] Available at: https://proceedings.neurips.cc/paper/2019/hash/f8eb278a8bce873ef365b45e939da38a-Abstract.html [Accessed 7 May 2023].
5. Outlier (2023) Wikipedia. Wikimedia Foundation. Available at: https://en.wikipedia.org/wiki/Outlier (Accessed: May 7, 2023). 
6. Pierre, S. (2022) A guide to time series analysis in Python, Built In. Available at: https://builtin.com/data-science/time-series-python (Accessed: May 7, 2023).